# Model
Use the given preprocessed input `X`, output `y`, evaluation functions `evaluate(y_pred, y_true)`, to train a chosen model (and save the model)  
Resulting model will be class named `Model` with methods:  
* fit(X, y) - train the model
* predict(X) - predict the output for given input
* save_model(path) - save the model to the given path
* load_model(path) - load the model from the given path
* evaluate() - evaluate the model performance

### KNN (K Nearest Neighbors)
https://scikit-learn.org/1.5/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import joblib

class KNN:
    def __init__(self, X, y, evaluate):
        self.evaluate_ = evaluate
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2)
        self.knn = KNeighborsClassifier(n_neighbors=10)
        
    def fit(self):
        self.knn.fit(self.X_train, self.y_train)

    def predict(self, X):
        return self.knn.predict(X)

    def evaluate(self):
        y_pred = self.predict(self.X_test)
        return self.evaluate_(y_pred, self.y_test)

    def save_model(self, path):
        joblib.dump(self.knn, path)

    def load_model(self, path):
        self.knn = joblib.load(path)

### Logistic Regression
https://scikit-learn.org/1.5/modules/generated/sklearn.linear_model.LogisticRegression.html

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression as LR
from sklearn.multioutput import MultiOutputClassifier

class LogisticRegression:
	def __init__(self, X, y, evaluate):
		self.evaluate_ = evaluate
		self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2)
		self.lr = MultiOutputClassifier(LR(penalty=None))

	def fit(self):
		self.lr.fit(self.X_train, self.y_train)
		
	def predict(self, X):
		return self.lr.predict(X)
	
	def evaluate(self):
		y_pred = self.predict(self.X_test)
		return self.evaluate_(y_pred, self.y_test)
	
	def save_model(self, path):
		joblib.dump(self.lr, path)

	def load_model(self, path):
		self.lr = joblib.load(path)


### Decision Trees
https://scikit-learn.org/1.5/modules/tree.html

In [ ]:
from sklearn import tree

class DecisionTree:
	def __init__(self, X, y, evaluate):
		self.evaluate_ = evaluate
		self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2)
		self.tree = tree.DecisionTreeClassifier()

	def fit(self):
		self.tree.fit(self.X_train, self.y_train)
		
	def predict(self, X):
		return self.tree.predict(X)
	
	def evaluate(self):
		y_pred = self.predict(self.X_test)
		return self.evaluate_(y_pred, self.y_test)
	
	def save_model(self, path):
		joblib.dump(self.tree, path)

	def load_model(self, path):
		self.tree = joblib.load(path)

### Random Forest

https://scikit-learn.org/1.5/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [ ]:
from sklearn.ensemble import RandomForestClassifier

class RandomForest:
	def __init__(self, X, y, evaluate):
		self.evaluate_ = evaluate
		self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2)
		self.rfc = RandomForestClassifier()

	def fit(self):
		self.rfc.fit(self.X_train, self.y_train)
		
	def predict(self, X):
		return self.rfc.predict(X)
	
	def evaluate(self):
		y_pred = self.predict(self.X_test)
		return self.evaluate_(y_pred, self.y_test)
	
	def save_model(self, path):
		joblib.dump(self.rfc, path)

	def load_model(self, path):
		self.rfc = joblib.load(path)

### Naive Bayes
https://scikit-learn.org/1.5/modules/naive_bayes.html

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.multioutput import MultiOutputClassifier

class NaiveBayes:
	def __init__(self, X, y, evaluate):
		self.evaluate_ = evaluate
		self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2)
		self.nb = MultiOutputClassifier(MultinomialNB())

	def fit(self):
		self.nb.fit(self.X_train, self.y_train)
		
	def predict(self, X):
		return self.nb.predict(X)
	
	def evaluate(self):
		y_pred = self.predict(self.X_test)
		return self.evaluate_(y_pred, self.y_test)
	
	def save_model(self, path):
		joblib.dump(self.nb, path)

	def load_model(self, path):
		self.nb = joblib.load(path)

### SVM (Support Vector Machine)
https://scikit-learn.org/1.5/modules/generated/sklearn.svm.SVC.html

In [ ]:
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier

class SVM:
    def __init__(self, X, y, evaluate):
        self.evaluate_ = evaluate
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2)
        self.svm = MultiOutputClassifier(SVC())

    def fit(self):
        self.svm.fit(self.X_train, self.y_train)
        
    def predict(self, X):
        return self.svm.predict(X)
    
    def evaluate(self):
        y_pred = self.predict(self.X_test)
        return self.evaluate_(y_pred, self.y_test)
    
    def save_model(self, path):
        joblib.dump(self.svm, path)

    def load_model(self, path):
        self.svm = joblib.load(path)

### Neural Network (Multi-layer Perceptron)
https://scikit-learn.org/1.5/modules/neural_networks_supervised.html

https://scikit-learn.org/dev/modules/generated/sklearn.neural_network.MLPClassifier.html

In [ ]:
from sklearn.neural_network import MLPClassifier

class MLP:
    def __init__(self, X, y, evaluate):
        self.evaluate_ = evaluate
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2)
        self.nn = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300)

    def fit(self):
        self.nn.fit(self.X_train, self.y_train)
        
    def predict(self, X):
        return self.nn.predict(X)
    
    def evaluate(self):
        y_pred = self.predict(self.X_test)
        return self.evaluate_(y_pred, self.y_test)
    
    def save_model(self, path):
        joblib.dump(self.nn, path)

    def load_model(self, path):
        self.nn = joblib.load(path)